# LaMa with Refinement - Google Colab

Uses your existing model from Google Drive with proper dependency management.

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Install PyTorch FIRST (before numpy)
# Using PyTorch 1.13.1 - oldest available that still supports refinement
!pip uninstall torch torchvision torchaudio -y
!pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
# Clone lama-with-refiner
!git clone https://github.com/geomagical/lama-with-refiner.git
%cd lama-with-refiner

In [ ]:
# Critical: Install numpy 1.21.6 for PyTorch 1.13
# LaMa needs numpy<1.24 but >=1.21 for PyTorch 1.13
!pip uninstall numpy -y
!pip install numpy==1.21.6

In [ ]:
# Install other dependencies with compatible versions for PyTorch 1.13
!pip install opencv-python==4.7.0.72 scikit-image==0.19.3 pyyaml==6.0 tqdm easydict==1.10
!pip install pytorch-lightning==1.5.10 kornia==0.6.12 hydra-core==1.3.2 omegaconf==2.3.0
!pip install tensorflow==2.11.0

In [ ]:
# Download perceptual loss model for refinement
!mkdir -p ade20k/ade20k-resnet50dilated-ppm_deepsup/
!wget -P ade20k/ade20k-resnet50dilated-ppm_deepsup/ http://sceneparsing.csail.mit.edu/model/pytorch/ade20k-resnet50dilated-ppm_deepsup/encoder_epoch_20.pth

In [ ]:
# Copy your model from Google Drive
!mkdir -p big-lama/models
!cp /content/drive/MyDrive/lama_models/best.ckpt big-lama/models/
!cp /content/drive/MyDrive/lama_models/config.yaml big-lama/

In [ ]:
# Upload and extract images
from google.colab import files
import zipfile
import shutil

uploaded = files.upload()
zip_name = list(uploaded.keys())[0]

!rm -rf input_images && mkdir -p input_images
with zipfile.ZipFile(zip_name, 'r') as zip_ref:
    zip_ref.extractall('input_images')

# Rename mask for LaMa format
import os
if os.path.exists('input_images/mask.png'):
    shutil.move('input_images/mask.png', 'input_images/image_mask000.png')

In [ ]:
# Run with refinement
!export TORCH_HOME=$(pwd) && export PYTHONPATH=$(pwd):$(pwd)/lama && \
python lama/bin/predict.py \
    model.path=$(pwd)/big-lama \
    indir=$(pwd)/input_images \
    outdir=$(pwd)/output_refined \
    refiner.enable=True \
    refiner.n_iters=15 \
    device=cuda

In [ ]:
# Display results
from PIL import Image
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

original = Image.open('input_images/image.png')
axes[0].imshow(original)
axes[0].set_title('Original')
axes[0].axis('off')

mask = Image.open('input_images/image_mask000.png').convert('L')
axes[1].imshow(mask, cmap='gray')
axes[1].set_title('Mask')
axes[1].axis('off')

result = Image.open('output_refined/image_mask000.png')
axes[2].imshow(result)
axes[2].set_title('Refined Result')
axes[2].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Download result
files.download('output_refined/image_mask000.png')